In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib as mp
import random as rd
import argparse
import os, sys
import csv
import math
import time
import matplotlib.pyplot as pl
class County:
    def __init__(self,parsename):
        self.parsename = parsename
        dataframe = xls.parse(parsename)
        self.data = dataframe
    def disp_all(self):
        print self.dataframe
    def get_all(self):
        return self.dataframe
xls = pd.ExcelFile('No-drybulb-dewpoint-short-dataset-CT.xlsx',header = None)
Zonal = []
Zonal.append(County('CT'))
ZonalNum = 1

In [2]:
# histo load based model stage 2
#num_epoches = 10000# training epoches for each customer samples
n_input_h = 63*24 # input size
input_seq_size_h = n_input_h
test_batch_size_h = 31 # days of a batch
valid_batch_size_h = 0
train_batch_size_h = 31
data_dim_h = 1 # same time of a week
n_output_h = 24
output_seq_size_h = 24
gap_h = 91
n_hidden_h_1= 30
n_hidden_h_2 =30
n_hidden_h_3 =30
tao_h = 0.5
bat = 91
# feature based model stage 1
#num_epoches = 30000 # training epoches for each customer samples
out_thresh = 18499
day_steps_f = 24
val_rate_f = 0.0
test_batch_size_f = test_batch_size_h*day_steps_f # days of a batch
valid_batch_size_f = valid_batch_size_h*day_steps_f
train_batch_size_f = train_batch_size_h*day_steps_f
n_output_f = 1
n_hidden_f_1 = 20
n_hidden_f_2 = 20
n_hidden_f_3 = 20
n_hidden_f_4 = 20
tao_f = 0.1
gap_test_f = 10
batch_size_f = test_batch_size_f # in this version, batch_size set same
preserve_f = 16115 ## amount of first time points without complete features
dropout_f = 0.9
data_norm = 0.0
# merge part stage 3
s3_input = day_steps_f
n_hidden_s3_1 = 60
n_hidden_s3_2 = 60
s3_output = 24
n_epochs = 3000

In [3]:
# DEMAND MATRIX 9 X LENGTH, 9: INC is total, index with 0, other substations are from 1 -> 8
rows_f = np.array(Zonal[0].data).shape[0]-0
columns_f = np.array(Zonal[0].data).shape[1]
database_f = np.zeros((ZonalNum, rows_f, columns_f))
for i in range(ZonalNum):
    tmp = np.array(Zonal[i].data)
    database_f[i] = tmp.astype(np.float32)[:,:]
database_f = np.transpose(database_f, [1, 0, 2])
totalen_f = rows_f
n_input_f = columns_f - 1
data_norm = np.max(database_f[:,:,0])
database_f[:,:,0] = database_f[:,:,0]/data_norm
db_f = database_f

rows = np.array(Zonal[0].data).shape[0]-0
columns = np.array(Zonal[0].data).shape[1]
# DEMAND MATRIX 9 X LENGTH, 9: INC is total, index with 0, other substations are from 1 -> 8
database_h = np.zeros((rows/output_seq_size_h, output_seq_size_h, columns))
tmp = np.array(Zonal[0].data)
tmp = tmp.astype(np.float32)[:,:]
database_h = tmp.reshape([rows/output_seq_size_h, output_seq_size_h, columns])
totalen_h = rows/output_seq_size_h
database_h[:,:,0] = database_h[:,:,0]/data_norm
db_h = database_h

In [4]:
#define id arrays
test_id_f = np.array(test_batch_size_f)
#valid_id_f = np.array(valid_batch_size_f)
#train_id_f = np.array(totalen_f - test_batch_size_f - valid_batch_size_f)

#give values to id arrays
#rang = range(preserve_f, totalen_f - test_batch_size_f)
#valid_id_f = rd.sample(rang,valid_batch_size_f)
test_id_f = np.array(range(totalen_f - test_batch_size_f,totalen_f))
#train_id_f = set(range(preserve_f, totalen_f - test_batch_size_f)) - set(valid_id_f)

#sort three id array
#valid_id_f = np.sort(valid_id_f)
test_id_f = np.sort(test_id_f)
#train_id_f = np.array(list(train_id_f))

#define id arrays
test_id_h = np.array(test_batch_size_h)
valid_id_h = np.array(valid_batch_size_h)
train_id_h = np.array(totalen_h-test_batch_size_h-valid_batch_size_h-input_seq_size_h)

id_start = preserve_f/24 + 1
#give values to id arrays
rang_h = range(input_seq_size_h/output_seq_size_h + id_start + gap_h,totalen_h-bat)
valid_id_h = rd.sample(rang_h,valid_batch_size_h)
test_id_h = np.array(range(totalen_h-test_batch_size_h,totalen_h))
train_id_h = set(range(input_seq_size_h/output_seq_size_h + gap_h + id_start,totalen_h-bat))-set(valid_id_h)

#sort three id array
valid_id_h = np.sort(valid_id_h)
test_id_h = np.sort(test_id_h)
train_id_h = np.array(list(train_id_h))
print test_id_h
print train_id_h

[3083 3084 3085 3086 3087 3088 3089 3090 3091 3092 3093 3094 3095 3096 3097
 3098 3099 3100 3101 3102 3103 3104 3105 3106 3107 3108 3109 3110 3111 3112
 3113]
[ 826  827  828 ..., 3020 3021 3022]


In [5]:
with tf.device('/cpu:0'):
    _X_f = tf.placeholder(tf.float32, [batch_size_f, ZonalNum, n_input_f], name = "X_f")
    _Y_f = tf.placeholder(tf.float32, [batch_size_f, ZonalNum, n_output_f], name = "Y_f")
    _Dropout_f = tf.placeholder(tf.float32, name = "Dropout_f")
    _X_h = tf.placeholder(tf.float32, [test_batch_size_h, n_input_h])
    _Y_h = tf.placeholder(tf.float32, [test_batch_size_h, n_output_h])

    # Create model
    def MLP_f(x, _dropout, weights, biases):    
        x = tf.reshape(x, [-1, n_input_f])
        # Hidden layer with RELU activation
        layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
        layer_1 = tf.nn.sigmoid(layer_1)
        #layer_1 = tf.nn.dropout(layer_1,_dropout)

        # Hidden layer with RELU activation
        layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
        layer_2 = tf.nn.sigmoid(layer_2)
        #layer_2 = tf.nn.dropout(layer_2,_dropout)

        # Hidden layer with RELU activation
        layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
        layer_3 = tf.nn.sigmoid(layer_3)
        #layer_3 = tf.nn.dropout(layer_3,_dropout)

        # Hidden layer with RELU activation
        layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
        layer_4 = tf.nn.sigmoid(layer_4)

        # Output layer with linear activation
        result = tf.matmul(layer_4, weights['out']) + biases['out']
        result = tf.nn.sigmoid(result)
        #result = tf.nn.dropout(result,_dropout)
        return result


    def MLP_h(x, weights, biases):
        # Hidden layer with RELU activation
        layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
        layer_1 = tf.nn.sigmoid(layer_1)
        #layer_1 = tf.nn.dropout(layer_1,_dropout)

        # Hidden layer with RELU activation
        layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
        layer_2 = tf.nn.sigmoid(layer_2)
        #layer_2 = tf.nn.dropout(layer_2,_dropout)

        # Hidden layer with RELU activation
        layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
        layer_3 = tf.nn.sigmoid(layer_3)
        #layer_3 = tf.nn.dropout(layer_3,_dropout)

        # Output layer with linear activation
        result = tf.matmul(layer_3, weights['out']) + biases['out']
        result = tf.nn.sigmoid(result)
        #result = tf.nn.dropout(result,_dropout)
        return result

    # MLP
    weights_f = {
        'h1': tf.Variable(tf.random_normal([n_input_f, n_hidden_f_1]), name = "w_f_1"),
        'h2': tf.Variable(tf.random_normal([n_hidden_f_1, n_hidden_f_2]), name = "w_f_2"),
        'h3': tf.Variable(tf.random_normal([n_hidden_f_2, n_hidden_f_3]), name = "w_f_3"),
        'h4': tf.Variable(tf.random_normal([n_hidden_f_3, n_hidden_f_4]), name = "w_f_4"),
        'out': tf.Variable(tf.random_normal([n_hidden_f_4, n_output_f]), name = "w_o")
    }
    biases_f = {
        'b1': tf.Variable(tf.random_normal([n_hidden_f_1]), name = "b_f_1"),
        'b2': tf.Variable(tf.random_normal([n_hidden_f_2]), name = "b_f_2"),
        'b3': tf.Variable(tf.random_normal([n_hidden_f_3]), name = "b_f_3"),
        'b4': tf.Variable(tf.random_normal([n_hidden_f_4]), name = "b_f_4"),
        'out': tf.Variable(tf.random_normal([n_output_f]), name = "b_o")
    }
    weights_h = {
        'h1': tf.Variable(tf.random_normal([n_input_h, n_hidden_h_1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_h_1, n_hidden_h_2])),
        'h3': tf.Variable(tf.random_normal([n_hidden_h_2, n_hidden_h_3])),
        'out': tf.Variable(tf.random_normal([n_hidden_h_3, n_output_h]))
    }
    biases_h = {
        'b1': tf.Variable(tf.random_normal([n_hidden_h_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_h_2])),
        'b3': tf.Variable(tf.random_normal([n_hidden_h_3])),
        'out': tf.Variable(tf.random_normal([n_output_h]))
    }
    p_f = MLP_f(_X_f, _Dropout_f, weights_f, biases_f)
    p_h = MLP_h(_X_h,  weights_h, biases_h)
    p_f = tf.reshape(p_f, [test_batch_size_h, s3_input])
    p_h = tf.reshape(p_h, [test_batch_size_h, s3_input])
    tao_s3 = tf.placeholder(tf.float32)
    _I = tf.concat(1, [p_f,p_h])
    _O = _Y_h
    weights_s3 = {
        'h1': tf.Variable(tf.random_normal([2*s3_input, n_hidden_s3_1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_s3_1, n_hidden_s3_2])),
        #'h3': tf.Variable(tf.random_normal([n_hidden_f_2, n_hidden_f_3])),
        #'h4': tf.Variable(tf.random_normal([n_hidden_f_3, n_hidden_f_4])),
        'out': tf.Variable(tf.random_normal([n_hidden_s3_2, s3_output]))
    }
    biases_s3 = {
        'b1': tf.Variable(tf.random_normal([n_hidden_s3_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_s3_2])),
        #'b3': tf.Variable(tf.random_normal([n_hidden_f_3])),
        #'b4': tf.Variable(tf.random_normal([n_hidden_f_4])),
        'out': tf.Variable(tf.random_normal([s3_output]))
    }
    def MLP_s3(x, weights, biases):
        # Hidden layer with RELU activation
        layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
        layer_1 = tf.nn.sigmoid(layer_1)
        #layer_1 = tf.nn.dropout(layer_1,_dropout)

        # Hidden layer with RELU activation
        layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
        layer_2 = tf.nn.sigmoid(layer_2)
        #layer_2 = tf.nn.dropout(layer_2,_dropout)
        # Output layer with linear activation
        result = tf.matmul(layer_2, weights['out']) + biases['out']
        result = tf.nn.sigmoid(result)
        #result = tf.nn.dropout(result,_dropout)
        return result

    _P = MLP_s3(_I, weights_s3, biases_s3)

    reshaped_outputs = tf.reshape(_P, [-1]) 

    reshaped_results = tf.reshape(_O,[-1])

        #cost function
    cost_s3 = 0.0
    for i in range(s3_input*test_batch_size_h):
        tmp = tf.cond(reshaped_results[i]-reshaped_outputs[i]>=0, lambda:(reshaped_results[i]-reshaped_outputs[i])*tao_s3,lambda:(reshaped_outputs[i]-reshaped_results[i])*(1.0-tao_s3))
        cost_s3 = tf.add(tmp,cost_s3)
    ccost_s3 = tf.reduce_mean(tf.pow(_O - _P,2)) 

    optimizer_s3 = tf.train.AdamOptimizer(learning_rate=0.001, beta1 = 0.8, beta2 = 0.7).minimize(ccost_s3)
    optimizer_s4 = tf.train.AdamOptimizer(learning_rate=0.0001, beta1 = 0.8, beta2 = 0.7).minimize(cost_s3)
    optimizer_s5 = tf.train.AdamOptimizer(learning_rate=0.00005, beta1 = 0.8, beta2 = 0.7).minimize(cost_s3)
    #n_hidden_s3_3 = 30
    #n_hidden_s3_4 = 30


def maxe(predictions, targets):
    return np.max(abs(predictions-targets))

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def mape(predictions, targets):
    return np.mean(abs(predictions-targets)/targets)
def variable_summaries(var, name):
    """Attach a lot of summaries to a Tensor."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.scalar_summary('mean/' + name, mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
        tf.scalar_summary('sttdev/' + name, stddev)
        tf.scalar_summary('max/' + name, tf.reduce_max(var))
        tf.scalar_summary('min/' + name, tf.reduce_min(var))
        tf.histogram_summary(name, var)

def test_data_gen():
    X_h = np.zeros((test_batch_size_h, n_input_h))
    Y_h = np.zeros((test_batch_size_h, n_output_h))
    X_f = np.zeros((test_batch_size_f,ZonalNum,n_input_f))
    Y_f = np.zeros((test_batch_size_f,ZonalNum,n_output_f))
    count_h = 0
    count_f = 0
    for i in test_id_f:
        Y_f[count_f] = db_f[i,:,:1]
        X_f[count_f] = db_f[i,:,1:]
        count_f = count_f + 1
    for i in test_id_h:
        Y_h[count_h,:] = (db_h[i,:,0])
        X_h[count_h,:] = (db_h[i-input_seq_size_h/output_seq_size_h-gap_h:i-gap_h,:,0]).reshape(n_input_h)
        count_h = count_h + 1
    X_h = X_h.astype(np.float32)
    Y_h = Y_h.astype(np.float32)
    X_f = X_f.astype(np.float32)
    Y_f = Y_f.astype(np.float32)
    return (X_h,Y_h,X_f,Y_f)

def train_data_gen():
    X_h = np.zeros((train_batch_size_h, n_input_h))
    Y_h = np.zeros((train_batch_size_h, n_output_h))
    X_f = np.zeros((train_batch_size_f,ZonalNum,n_input_f))
    Y_f = np.zeros((train_batch_size_f,ZonalNum,n_output_f))
    count_h = 0
    count_f = 0
    rang = range(0,train_id_h.shape[0])
    train_rd = rd.sample(rang,train_batch_size_h)
    train_rd = np.sort(train_rd)
    for i in train_rd:
        k = train_id_h[i]
        Y_h[count_h] = db_h[k,:,0]
        X_h[count_h] = (db_h[k-input_seq_size_h/output_seq_size_h-gap_h:k-gap_h,:,0]).reshape(n_input_h)
        for j in range(24):
            Y_f[count_f] = db_f[k*24+j,:,:1]
            X_f[count_f] = db_f[k*24+j,:,1:]
            count_f = count_f + 1
        count_h = count_h + 1
    X_h = X_h.astype(np.float32)
    Y_h = Y_h.astype(np.float32)
    X_f = X_f.astype(np.float32)
    Y_f = Y_f.astype(np.float32)
    return (X_h,Y_h,X_f,Y_f)

def valid_data_gen():
    X_h = np.zeros((valid_batch_size_h, n_input_h))
    Y_h = np.zeros((valid_batch_size_h, n_output_h))
    X_f = np.zeros((valid_batch_size_f,ZonalNum,n_input_f))
    Y_f = np.zeros((valid_batch_size_f,ZonalNum,n_output_f))
    count_h = 0
    count_f = 0
    rang = range(0,valid_id_h.shape[0])
    valid_rd = rd.sample(rang,train_batch_size_h)
    valid_rd = np.sort(valid_rd)
    for i in train_rd:
        k = valid_id_h[i]
        Y_h[count_h] = db_h[k,:,0]
        X_h[count_h] = (db_h[k-input_seq_size_h/output_seq_size_h-gap_h:k-gap_h,:,0]).reshape(n_input_h)
        for j in range(24):
            Y_f[count_f] = db_f[k*24+j,:,:1]
            X_f[count_f] = db_f[k*24+j,:,1:]
            count_f = count_f + 1
        count_h = count_h + 1
    X_h = X_h.astype(np.float32)
    Y_h = Y_h.astype(np.float32)
    X_f = X_f.astype(np.float32)
    Y_f = Y_f.astype(np.float32)
    return (X_h,Y_h,X_f,Y_f)

In [ ]:
init = tf.initialize_all_variables()
taolist = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
time1 = time.time()
for t in xrange(0,9):
    tmp_tao = taolist[t]
    saver = tf.train.Saver({"wf1":weights_f['h1'],"wf2":weights_f['h2'],"wf3":weights_f['h3'],"wf4":weights_f['h4'],"wfo":weights_f['out'],"bf1":biases_f['b1'],"bf2":biases_f['b2'],"bf3":biases_f['b3'],"bf4":biases_f['b4'],"bfo":biases_f['out'],'wh1':weights_h['h1'],'wh2':weights_h['h2'],'wh3':weights_h['h3'],'who':weights_h['out'],'bh1':biases_h['b1'],'bh2':biases_h['b2'],'bh3':biases_h['b3'],'bho':biases_h['out'],"wf1":weights_f['h1'],"wf2":weights_f['h2'],"wf3":weights_f['h3'],"wf4":weights_f['h4'],"wfo":weights_f['out'],"bf1":biases_f['b1'],"bf2":biases_f['b2'],"bf3":biases_f['b3'],"bf4":biases_f['b4'],"bfo":biases_f['out'],'wh1':weights_h['h1'],'wh2':weights_h['h2'],'wh3':weights_h['h3'],'who':weights_h['out'],'bh1':biases_h['b1'],'bh2':biases_h['b2'],'bh3':biases_h['b3'],'bho':biases_h['out'],"ws1":weights_s3['h1'],"ws2":weights_s3['h2'],"wso":weights_s3['out'],'bs1':biases_s3['b1'],'bs2':biases_s3['b2'],'bso':biases_s3['out']})
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    sess.run(init)
    saver.restore(sess,"C-models/model-ME-rmse-pinball-"+str(tmp_tao)+ "-checkpoint-l4-hs.ckpt")
    
    costs_train = []
    costs_test = []
    predlist = []
    for i in range(n_epochs):
        # training process
        x_h,y_h,x_f,y_f = train_data_gen()
        y_s3 = y_h
        _ = sess.run([optimizer_s5], feed_dict = {_X_f: x_f, _Y_f: y_f, _X_h: x_h, _Y_h: y_h, tao_s3: tmp_tao})
        if i % 100 == 0:
            x_h,y_h,x_f,y_f = train_data_gen()
            P,  err_t, err_c = sess.run([_P, cost_s3,ccost_s3], feed_dict = {_X_f: x_f, _Y_f: y_f, _X_h: x_h, _Y_h: y_h, tao_s3: tmp_tao})
            print "Iter " + str(i) + ", Minibatch Loss ---- Pinball = " + str(err_t) + "   ****   RMSE = " + str(np.sqrt(err_c))
            P = np.reshape(np.array(P),[-1])
            costs_test.append(err_t)
            predlist.append(P)
        if i == n_epochs-1:
            x_h,y_h,x_f,y_f = train_data_gen()
            P,  err_t, err_c = sess.run([_P, cost_s3,ccost_s3], feed_dict = {_X_f: x_f, _Y_f: y_f, _X_h: x_h, _Y_h: y_h, tao_s3: tmp_tao})
            print "Iter " + str(i) + ", Minibatch Loss ---- Pinball = " + str(err_t) + "   ****   RMSE = " + str(np.sqrt(err_c))
            P = np.reshape(np.array(P),[-1])
            costs_test.append(err_t)
            predlist.append(P)
    saver.save(sess,"C-models/model-ME-rmse-pinball-"+str(tmp_tao)+ "-checkpoint-l5-hs.ckpt")
    print "C-models/model-ME-rmse-pinball-"+str(tmp_tao)+ "-checkpoint-l5-hs.ckpt"

In [6]:
"""
init = tf.initialize_all_variables()
taolist = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
time1 = time.time()
for t in xrange(1,9):
    tmp_tao = taolist[t]
    saver = tf.train.Saver({"wf1":weights_f['h1'],"wf2":weights_f['h2'],"wf3":weights_f['h3'],"wf4":weights_f['h4'],"wfo":weights_f['out'],"bf1":biases_f['b1'],"bf2":biases_f['b2'],"bf3":biases_f['b3'],"bf4":biases_f['b4'],"bfo":biases_f['out'],'wh1':weights_h['h1'],'wh2':weights_h['h2'],'wh3':weights_h['h3'],'who':weights_h['out'],'bh1':biases_h['b1'],'bh2':biases_h['b2'],'bh3':biases_h['b3'],'bho':biases_h['out'],"wf1":weights_f['h1'],"wf2":weights_f['h2'],"wf3":weights_f['h3'],"wf4":weights_f['h4'],"wfo":weights_f['out'],"bf1":biases_f['b1'],"bf2":biases_f['b2'],"bf3":biases_f['b3'],"bf4":biases_f['b4'],"bfo":biases_f['out'],'wh1':weights_h['h1'],'wh2':weights_h['h2'],'wh3':weights_h['h3'],'who':weights_h['out'],'bh1':biases_h['b1'],'bh2':biases_h['b2'],'bh3':biases_h['b3'],'bho':biases_h['out'],"ws1":weights_s3['h1'],"ws2":weights_s3['h2'],"wso":weights_s3['out'],'bs1':biases_s3['b1'],'bs2':biases_s3['b2'],'bso':biases_s3['out']})
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    sess.run(init)
    saver.restore(sess,"C-models/model-ME-rmse-pinball-"+str(tmp_tao)+ "-checkpoint-k-hs.ckpt")
    
    costs_train = []
    costs_test = []
    predlist = []
    for i in range(n_epochs):
        # training process
        x_h,y_h,x_f,y_f = train_data_gen()
        y_s3 = y_h
        _ = sess.run([optimizer_s3], feed_dict = {_X_f: x_f, _Y_f: y_f, _X_h: x_h, _Y_h: y_h, tao_s3: tmp_tao})
        if i % 100 == 0:
            x_h,y_h,x_f,y_f = train_data_gen()
            P,  err_t, err_c = sess.run([_P, cost_s3,ccost_s3], feed_dict = {_X_f: x_f, _Y_f: y_f, _X_h: x_h, _Y_h: y_h, tao_s3: tmp_tao})
            print "Iter " + str(i) + ", Minibatch Loss ---- Pinball = " + str(err_t) + "   ****   RMSE = " + str(np.sqrt(err_c))
            P = np.reshape(np.array(P),[-1])
            costs_test.append(err_t)
            predlist.append(P)
        if i == n_epochs-1:
            x_h,y_h,x_f,y_f = train_data_gen()
            P,  err_t, err_c = sess.run([_P, cost_s3,ccost_s3], feed_dict = {_X_f: x_f, _Y_f: y_f, _X_h: x_h, _Y_h: y_h, tao_s3: tmp_tao})
            print "Iter " + str(i) + ", Minibatch Loss ---- Pinball = " + str(err_t) + "   ****   RMSE = " + str(np.sqrt(err_c))
            P = np.reshape(np.array(P),[-1])
            costs_test.append(err_t)
            predlist.append(P)
    saver.save(sess,"C-models/model-ME-rmse-pinball-"+str(tmp_tao)+ "-checkpoint-l1-hs.ckpt")
    print "C-models/model-ME-rmse-pinball-"+str(tmp_tao)+ "-checkpoint-l1-hs.ckpt"
    
    costs_train = []
    costs_test = []
    predlist = []
    for i in range(n_epochs):
        # training process
        x_h,y_h,x_f,y_f = train_data_gen()
        y_s3 = y_h
        _ = sess.run([optimizer_s4], feed_dict = {_X_f: x_f, _Y_f: y_f, _X_h: x_h, _Y_h: y_h, tao_s3: tmp_tao})
        if i % 100 == 0:
            x_h,y_h,x_f,y_f = train_data_gen()
            P,  err_t, err_c = sess.run([_P, cost_s3,ccost_s3], feed_dict = {_X_f: x_f, _Y_f: y_f, _X_h: x_h, _Y_h: y_h, tao_s3: tmp_tao})
            print "Iter " + str(i) + ", Minibatch Loss ---- Pinball = " + str(err_t) + "   ****   RMSE = " + str(np.sqrt(err_c))
            P = np.reshape(np.array(P),[-1])
            costs_test.append(err_t)
            predlist.append(P)
        if i == n_epochs-1:
            x_h,y_h,x_f,y_f = train_data_gen()
            P,  err_t, err_c = sess.run([_P, cost_s3,ccost_s3], feed_dict = {_X_f: x_f, _Y_f: y_f, _X_h: x_h, _Y_h: y_h, tao_s3: tmp_tao})
            print "Iter " + str(i) + ", Minibatch Loss ---- Pinball = " + str(err_t) + "   ****   RMSE = " + str(np.sqrt(err_c))
            P = np.reshape(np.array(P),[-1])
            costs_test.append(err_t)
            predlist.append(P)
    saver.save(sess,"C-models/model-ME-rmse-pinball-"+str(tmp_tao)+ "-checkpoint-l2-hs.ckpt")
    print "C-models/model-ME-rmse-pinball-"+str(tmp_tao)+ "-checkpoint-l2-hs.ckpt"
    
    costs_train = []
    costs_test = []
    predlist = []
    for i in range(n_epochs):
        # training process
        x_h,y_h,x_f,y_f = train_data_gen()
        y_s3 = y_h
        _ = sess.run([optimizer_s5], feed_dict = {_X_f: x_f, _Y_f: y_f, _X_h: x_h, _Y_h: y_h, tao_s3: tmp_tao})
        if i % 100 == 0:
            x_h,y_h,x_f,y_f = train_data_gen()
            P,  err_t, err_c = sess.run([_P, cost_s3,ccost_s3], feed_dict = {_X_f: x_f, _Y_f: y_f, _X_h: x_h, _Y_h: y_h, tao_s3: tmp_tao})
            print "Iter " + str(i) + ", Minibatch Loss ---- Pinball = " + str(err_t) + "   ****   RMSE = " + str(np.sqrt(err_c))
            P = np.reshape(np.array(P),[-1])
            costs_test.append(err_t)
            predlist.append(P)
        if i == n_epochs-1:
            x_h,y_h,x_f,y_f = train_data_gen()
            P,  err_t, err_c = sess.run([_P, cost_s3,ccost_s3], feed_dict = {_X_f: x_f, _Y_f: y_f, _X_h: x_h, _Y_h: y_h, tao_s3: tmp_tao})
            print "Iter " + str(i) + ", Minibatch Loss ---- Pinball = " + str(err_t) + "   ****   RMSE = " + str(np.sqrt(err_c))
            P = np.reshape(np.array(P),[-1])
            costs_test.append(err_t)
            predlist.append(P)
    saver.save(sess,"C-models/model-ME-rmse-pinball-"+str(tmp_tao)+ "-checkpoint-l3-hs.ckpt")
    print "C-models/model-ME-rmse-pinball-"+str(tmp_tao)+ "-checkpoint-l3-hs.ckpt"
    
    costs_train = []
    costs_test = []
    predlist = []
    for i in range(n_epochs):
        # training process
        x_h,y_h,x_f,y_f = train_data_gen()
        y_s3 = y_h
        _ = sess.run([optimizer_s5], feed_dict = {_X_f: x_f, _Y_f: y_f, _X_h: x_h, _Y_h: y_h, tao_s3: tmp_tao})
        if i % 100 == 0:
            x_h,y_h,x_f,y_f = train_data_gen()
            P,  err_t, err_c = sess.run([_P, cost_s3,ccost_s3], feed_dict = {_X_f: x_f, _Y_f: y_f, _X_h: x_h, _Y_h: y_h, tao_s3: tmp_tao})
            print "Iter " + str(i) + ", Minibatch Loss ---- Pinball = " + str(err_t) + "   ****   RMSE = " + str(np.sqrt(err_c))
            P = np.reshape(np.array(P),[-1])
            costs_test.append(err_t)
            predlist.append(P)
        if i == n_epochs-1:
            x_h,y_h,x_f,y_f = train_data_gen()
            P,  err_t, err_c = sess.run([_P, cost_s3,ccost_s3], feed_dict = {_X_f: x_f, _Y_f: y_f, _X_h: x_h, _Y_h: y_h, tao_s3: tmp_tao})
            print "Iter " + str(i) + ", Minibatch Loss ---- Pinball = " + str(err_t) + "   ****   RMSE = " + str(np.sqrt(err_c))
            P = np.reshape(np.array(P),[-1])
            costs_test.append(err_t)
            predlist.append(P)
    saver.save(sess,"C-models/model-ME-rmse-pinball-"+str(tmp_tao)+ "-checkpoint-l4-hs.ckpt")
    print "C-models/model-ME-rmse-pinball-"+str(tmp_tao)+ "-checkpoint-l4-hs.ckpt"
"""

Iter 0, Minibatch Loss ---- Pinball = 10.6495   ****   RMSE = 0.0386468
Iter 100, Minibatch Loss ---- Pinball = 11.2162   ****   RMSE = 0.0345703
Iter 200, Minibatch Loss ---- Pinball = 7.95433   ****   RMSE = 0.0620664
Iter 300, Minibatch Loss ---- Pinball = 11.2144   ****   RMSE = 0.0553146
Iter 400, Minibatch Loss ---- Pinball = 12.0789   ****   RMSE = 0.0438039
Iter 500, Minibatch Loss ---- Pinball = 10.2298   ****   RMSE = 0.0369959
Iter 600, Minibatch Loss ---- Pinball = 11.7984   ****   RMSE = 0.0377777
Iter 700, Minibatch Loss ---- Pinball = 9.12281   ****   RMSE = 0.0384549
Iter 800, Minibatch Loss ---- Pinball = 13.2534   ****   RMSE = 0.0336857
Iter 900, Minibatch Loss ---- Pinball = 11.7963   ****   RMSE = 0.0310756
Iter 1000, Minibatch Loss ---- Pinball = 6.40597   ****   RMSE = 0.0405673
Iter 1100, Minibatch Loss ---- Pinball = 15.1454   ****   RMSE = 0.0530586
Iter 1200, Minibatch Loss ---- Pinball = 18.0656   ****   RMSE = 0.046238
Iter 1300, Minibatch Loss ---- Pinball

In [7]:
rmseList = []
mapeList = []
x_h,y_h,x_f,y_f = test_data_gen()
acload = np.reshape(np.array(y_h), [-1])
for prload in predlist:
    prload = np.reshape(np.array(prload), [-1])
    rmseList.append(rmse(prload,acload))
    mapeList.append(mape(prload,acload))
print "Final model RMSE = " + str(np.mean(rmseList[-11:-1]))
print rmseList[-11:]
print "Final model MAPE = " + str(np.mean(mapeList[-11:-1]))
print mapeList[-11:]
print time.time()-time1

Final model RMSE = 0.549552
[0.56295753, 0.54707646, 0.54602301, 0.55467844, 0.54151911, 0.5507791, 0.55399346, 0.5415321, 0.53306204, 0.56390256, 0.56950492]
Final model MAPE = inf
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
12281.6526141


In [8]:
'''
l = tf.placeholder(tf.float32, [24*31])
r = tf.placeholder(tf.float32, [24*31])
cost_s3 = tf.Variable(initial_value = 0.0)
    #cost function
for i in range(24*31):
    tmp = tf.cond(l[i]-r[i]>=0, lambda:(l[i]-r[i])*0.1,lambda:(r[i]-l[i])*(1.0-0.1))
    cost_s3 = tf.add(tmp,cost_s3)
cost_s3 = cost_s3/31
    
init = tf.initialize_all_variables()
sess1 = tf.Session()
sess1.run(init)
print sess1.run(cost_s3, feed_dict = {l:P,r:y_f})
'''

'\nl = tf.placeholder(tf.float32, [24*31])\nr = tf.placeholder(tf.float32, [24*31])\ncost_s3 = tf.Variable(initial_value = 0.0)\n    #cost function\nfor i in range(24*31):\n    tmp = tf.cond(l[i]-r[i]>=0, lambda:(l[i]-r[i])*0.1,lambda:(r[i]-l[i])*(1.0-0.1))\n    cost_s3 = tf.add(tmp,cost_s3)\ncost_s3 = cost_s3/31\n    \ninit = tf.initialize_all_variables()\nsess1 = tf.Session()\nsess1.run(init)\nprint sess1.run(cost_s3, feed_dict = {l:P,r:y_f})\n'

In [9]:
'''
ac = np.array(acload)
prload = predlist[-1]
pr = np.array(np.reshape(np.array(prload), [-1]))
t = np.array(range(ac.shape[0]))

label_f0 = r"real load"
label_f1 = r"predicted load"

p1 = pl.subplot()
p1.plot(t,ac,"g",label=label_f0,linewidth=2)
p1.plot(t,pr,"r:",label=label_f1,linewidth=2)

#p1.axis([0,700,10000,25000])

p1.set_ylabel("Demand (kWh)",fontsize=14)
p1.set_xlabel("Time Point",fontsize=14)
#p1.set_title("A simple example",fontsize=18)
p1.grid(True)
p1.legend()

pl.show()
"""

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-9-351880c9d799>, line 23)